In [ ]:
# Importing Libraries
import os
import pandas as pd

# OpenAI LLM
from langchain_openai import ChatOpenAI

# Document Loading
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Environment vaiables
from dotenv import load_dotenv

In [2]:
# Load your API key from a .env file
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Loading PDF
pdf_path = 'Resources/Resume - Tin Pham - Dec 2024.pdf'

loader = PyPDFLoader(pdf_path)
pages = loader.load()

Ignoring wrong pointing object 118 0 (offset 0)


In [5]:
# Split the text into management chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, # This ensures each chunk is within the token limit for GPT models
    chunk_overlap = 50
)

chunks = splitter.split_documents(pages)

In [6]:
# Convert chunks into a Pandas Dataframe
df_chunks = pd.DataFrame([{
    'chunk_id': i,
    'content': chunk.page_content.strip()
} for i, chunk in enumerate(chunks)])

df_chunks.head()

,chunk_id,content
0,0,"617 Longhorn Cavern Rd., Leander, TX 786..."
1,1,▪ \n \nCollaborated\n \nextensively\n \nwith\n...
2,2,July 2019 – February 2022 Goog...
3,3,processes\n \nwith\n the extended Sales tea...
4,4,accounts\n \nin\n \nSouth\n \nCentral\n \nfor\...


In [7]:
# Saving DF to CSV
df_chunks.to_csv('pdf_chunks.csv', index=False)

In [ ]:
# Initialize the Chat Model
llm = ChatOpenAI(openai_api_key = openai_key, temperature = 0)

C:\Users\flash\AppData\Local\Temp\ipykernel_41832\3284031262.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key = openai_key, temperature = 0)
